# Using OpenAI with LocalAI (fully free)
You can use [OpenAI](https://github.com/openai/openai-python) with [LocalAI](https://github.com/go-skynet/LocalAI) (fully free) in your local machine.
## Requirements
You will need a minimum of 150GB disk space, 16 GB RAM, and a 64-bit processor.

## LocalAI setup (docker)

### Docker setup

- Install [docker](https://docs.docker.com/engine/install/).
- Optional: set docker data folder to use another disk (partition) to avoid system partition collapse:

In [ ]:
sudo nano /etc/docker/daemon.json
# Add the "data-root" & save (NOTE: REPLACE /mnt/sda5 with your HDD partition)       
{
    "data-root": "/mnt/sda5/docker/data-root"
}
# Restart docker service
sudo systemctl restart docker.service

- Create a volume for the models (this avoid LocalAI download the models every time we start the docker container)
- Run the LocalAI container (first time will install the container & download the models).

In [ ]:
docker volume create localai-models
docker run -p 8080:8080 --name local-ai -ti -v localai-models:/build/models localai/localai:latest-aio-cpu

## Cursor (VSCode AI IDE) with LocalAI
~~Just set the local host/port & a fake api_key:~~ DONT WORK WITH OLLAMA

## Ollama (local setup)
Using local Ollama with installer in linux (as system service).
You will need a compatible GPU hardware to make it run reasonably fast.
#### Install 
- Go to the [Ollama](https://ollama.com/download) web site and use the curl + sh installer
- Ensure service is working: `sudo service ollama status`
- See logs: `sudo journalctl -u ollama -f`
  - Checkout is using GPU (not CPU)
  - You can also see the used VRAM % (dynamic) in Nvidia Settings application
#### Run a model, depending on your hardware (GPU, CPU, etc)
In my case I've tested local Ollama with the following hardware:
- Intel® Core™ i7-6700HQ CPU @ 2.60GHz × 8
- NVIDIA GeForce GTX 960M (2MB)
- Ubuntu 22.04.5 LTS
Before installing check you have enough space in your HDD.  [Ollama3.1:8b](https://ollama.com/library/llama3.1) weights 4.7GB.

If you don't have enough space or want to store models in another disk partition see below: [Change Ollama models store folder](#change-ollama-models-store-folder)
```bash
  ollama run llama3.1:8b #recommended
  #ollama run tinyllama
  #ollama run llama2
  ```
### Troubleshooting
[Ollama docs/troubleshooting](https://github.com/ollama/ollama/blob/main/docs/troubleshooting.md)
#### Change Ollama models store folder
Change ollama models folder to avoid system partition saturation.
[Models](https://ollama.com/library?sort=popular) are big, so I've changed the location folder for Ollama models:
##### Create new folder (and move models)
```bash
data_folder=/mnt/sda5/ollama \
&& sudo mkdir -p "${data_folder}"/models \
&& sudo chown -R ollama:ollama "${data_folder}"
```
If you already have downloaded models, move de folder to the new location
```bash
mv /usr/share/ollama/.ollama/models/ /mnt/sda5/ollama
```
##### Set the OLLAMA_MODELS env variable
> Didn't work sudo systemctl edit ollama so edited ollama.service directly

> Didn't work export OLLAMA_MODELS in ~/.zshrc did not work when restarting ollama
Change ollama.service
```bash
sudo nano /etc/systemd/system/ollama.service
# add this line at the end of [Service] block
Environment="OLLAMA_MODELS=/mnt/sda5/ollama/models"
#save and exit
sudo systemctl daemon-reload
sudo systemctl restart ollama
```

#### Ollama don't run with CPU after Linux systems sleep->wake up
```bash
sudo systemctl stop ollama
sudo rmmod nvidia_uvm && sudo modprobe nvidia_uvm
sudo systemctl start ollama
```

  

## Langchain -> OpenAI -> LocalAI

### Project setup
#### Prerequisites

[Install](https://docs.anaconda.com/miniconda/#quick-command-line-install) Conda. (See also: [Conda getting started](https://conda.io/projects/conda/en/latest/user-guide/getting-started.html#before-you-start))
```bash
eval "$(/home/slks/miniconda3/bin/conda shell.bash hook)"
```

#### Install dependencies
[Activate environment](https://conda.io/projects/conda/en/latest/user-guide/getting-started.html#installing-packages) to install dependencies:


In [ ]:
conda activate ./.venv

In [ ]:
# # Create virtual environment  (ALSO with conda)
# python -m venv myenv
# # Activate virtual environment
# source myenv/bin/activate
# # Install dependencies
# pip install langchain
# pip install langchain-openai

In [ ]:
# some conda commands used to setup project
conda init zsh
conda info --envs
conda init langchain 
conda create --name langchain
conda activate langchain
conda config --add channels conda-forge
conda config --set channel_priority strict


We'll use LocalAI host & port, the api key is required by the OpenAI library only, just put any value.

In [2]:
from host import hostArgs
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(**hostArgs)
llm.invoke("La profesión de ingeniero de IA, tiene un futuro prometedor?")
# client.chat.completions.create(model="gpt-4",messages=[{"role":"user","content":"La profesión de ingeniero de software, tiene un futuro prometedor?"}])
# chatCompletion = client.chat.completions.create(model="gpt-4",messages=[{"role":"user","content":"La profesión de ingeniero de software, tiene un futuro prometedor?"}])
# print(chatCompletion.choices[0].message.content)
# chatCompletion.create(messages=[{"role":"user","content":"Como dices?"}], chat_id=chatCompletion.id)

AIMessage(content='La profesión de ingeniero en Inteligencia Artificial (IA) tiene efectivamente un buen número de oportunidades y potencial para expandirse, debido a varias razones.\n\nLas posibilidades en el campo del engenyeo de algoritmis, son:\n\n- **Diversidad. Las áreas disciplinarias han sido incorporadas**.\nEs poco común ver que los ingenieros especializados en inteligencia artificial también podrían abarcarse una variedad de conocimientos o intereses adicionales que les permiten entrar campos como robótica, visión artificial y más.\n\nAdemás esto último no será algo nuevo, debido a la cada vez mayor incorporación entre todos los campos.\n- **Reemplazo humanodigital, por ejemplo en atención al cliente** en los procesos de call-center.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 169, 'prompt_tokens': 26, 'total_tokens': 195}, 'model_name': 'llama3.1:8b', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': N

In [ ]:
from host import hostArgs
from langchain.llms import OpenAI

llm = OpenAI(**hostArgs)
text = "What would be a good company name for a company that makes colorful socks?"
print(llm(text))

## Memory

In [4]:
from host import hostArgs
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

from langchain_openai.chat_models import ChatOpenAI

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

model = ChatOpenAI(**hostArgs)
prompt = ChatPromptTemplate.from_messages([
        ("system","Eres un asistente especializado en {ability}.  Puedes responder preguntas sobre {ability}"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}")
        ])
runnable = prompt | model

with_message_history = RunnableWithMessageHistory(runnable, get_session_history, input_messages_key="input", history_messages_key="history")
with_message_history.invoke(
    input={"history": "history1", "ability": "Ciencia de datos", "input": "Qué es una función de perdida?"},
    config={"configurable": {"session_id": "session_1"}})
#           CPU  GPU (GTX960M)
#tinyllama  ?:?? 2:56
#llama2     2:24 1:34 (llama2 with host model llama3.1)
#llama3.1   5:40 2:10

AIMessage(content='¡Buena pregunta!\n\nEn aprendizaje automático, una función de pérdida (loss function) es un parámetro matemátificado utilizado para cuantificar el desempeño de un modelo prevista contra las observaciones reales. Su objetivo principal es minimizar la diferencia entre las predicciones del modelo y los resultados observados.\n\nEn otras palabras, la función de pérdida medidas la distancia o disímulo existente entre los salidos previstas por el algoritmos (predictores) y las realdes observadas(pobres data disponibles). Por lo general , estos números se calculan usando datos conexas y entonces usandose para estimir la capacidad correcta del Algoritmos. \n\n\nAlgunos ejemplos de funciones de pérdida comunes son:\n\n- **MSE (Mean Squared Error)**: esta función de pérdida es también conocida como cuadrática o mean absolute deviation. Esto es cuando el error medio en las prediccion se calcular como la magnitud media, es decir el promedio en cueros números de predicción.\n\nEj

In [6]:
with_message_history.invoke(
    input={"history": "history1", "ability": "Ciencia de datos", "input": "para un ingeniero de software neofito en este campo, explicame más sobre las bases de ciencia de datos y redes neuronales"},
    config={"configurable": {"session_id": "session_1"}}
)
#          CPU   GPU (GTX960M)
#tinyllama 11:38
#llama2    12:00 3:31 (llama2 with host model llama3.1)
#llama3.1  16:00 7:04 (9:29 with firefox opended)

AIMessage(content='¡Claro!\n\nImagina que tienes una caja llena de juguetes que te gustan de diversa naturaleza. Podrías contarlos (número de juguetes) o agruparlos por diferentes categorías como:\n\n- Juguetesp por temas. (**Categorizando**): Aquí está el primer paso para comprernder conceptaos\n- Edad de niños por lo que cada juegos puede llegar a estar muy amplio.\n- Colores\n\n Ahora tienes más información:\n\nJuguetesp están organizados en una forma determinada. En las diversas etapas y tipos como pueden: categorizando un modelo de datos.\n\n\nPídelos.\n\nImagina, estas etiquetas con distintas variables que se han identificado con lo datos como por ejemplo:\n\n\n- Edad del hijo\n- Correo y género\n\nTienes algo adicional. Imáxim que este un conjunto amplio en la caja .\n\n\nPara esta explicaciones tienes más de esta forma y los diferentes concepto que tenemos el tener al data set.\n\n\n¡Un paso hacia!\n\nPara entender cada uno, podemos compararlo a una categoría muy útil y de usos